In [2]:
import pandas as pd
import numpy as np
import os, zipfile
import subprocess

In [41]:
# compare standalone models for binary classification
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from imblearn.ensemble import BalancedBaggingClassifier
import lightgbm as lgbm
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
import lightgbm as lgbm
from sklearn.ensemble import StackingClassifier
from matplotlib import pyplot

In [24]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

In [25]:
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split, GridSearchCV, RandomizedSearchCV

In [15]:
from imblearn.under_sampling import RandomUnderSampler

In [3]:
import category_encoders as ce

In [6]:
dir_name = 'D:\\1mfti\ml\Home-Credit-Default-Risk'
os.chdir(dir_name)
train = pd.read_csv('data/application_train.csv')
test = pd.read_csv('data/application_test.csv')

In [9]:
num_cols = list(train.select_dtypes(include=['int64', 'float64']).columns)
num_cols.remove('TARGET')

In [12]:
cat_cols = list(train.select_dtypes(include=['object']).columns)

In [16]:
def preprocess(train1, test1):
    train1 = train1.fillna(-10)
    test1 = test1.fillna(-10)
    train1[num_cols] = train1[num_cols].apply(lambda x: pd.cut(x, bins=4, labels=False))
    test1[num_cols] = test1[num_cols].apply(lambda x: pd.cut(x, bins=4, labels=False))
    WOE_encoder = ce.WOEEncoder()
    train1[cat_cols] = WOE_encoder.fit_transform(train1[cat_cols], train1['TARGET'])
    test1[cat_cols] = WOE_encoder.transform(train1[cat_cols])

    return train1, test1

In [73]:
train_norm, test_norm = preprocess(train, test)

C:\Python\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [74]:
X_train = train_norm.drop('TARGET', axis=1)
y_train = train_norm['TARGET']
X_test = test_norm

In [75]:
ada = RandomUnderSampler(random_state=42)
X_res, y_res = ada.fit_resample(X_train, y_train)

In [29]:
# X_res, y_res = X_res[:20000], y_res[:20000]

In [76]:
X_res, X_valid, y_res, y_valid = train_test_split(X_res, y_res,
                                                    stratify=y_res, 
                                                    test_size=0.2)

In [65]:
lgbm_clf = lgbm.LGBMClassifier(objective='binary', 
         boosting_type = 'goss', 
         n_estimators = 50, class_weight ='balanced', learning_rate =0.01, random_state=42, eval_set=[X_valid, y_valid])

In [66]:
lgbm_clf.fit(X_res, y_res)

LGBMClassifier(boosting_type='goss', class_weight='balanced',
               eval_set=[        SK_ID_CURR  NAME_CONTRACT_TYPE  CODE_GENDER  FLAG_OWN_CAR  \
143886           1            0.036202    -0.154315      0.056221   
12347            0            0.036202     0.250941     -0.117305   
6867             0            0.036202    -0.154315     -0.117305   
93346            1            0.036202     0.250941      0.056221   
182281           2            0.036202    -0.154315      0.056221   
...            ...                 ...          ...           ...   
167301           2            0.036202    -0.154315     -0.117305   
50257            0            0.036202    -0.154315     -...
182281                          1                          0   
...                           ...                        ...   
167301                          1                          0   
50257                           1                          0   
171000                          1           

In [ ]:
# Точность на kaggle:0.57896 learning_rate=0.01, n_estimators=50, objective='binary',
               random_state=42)

In [67]:
lgbm_clf.importance

AttributeError: 'LGBMClassifier' object has no attribute 'importance'

In [30]:
random_state=42

fit_params = {"early_stopping_rounds" : 100,
              "eval_metric" : 'auc',
              "eval_set" : [(X_res,y_res)],
              'eval_names': ['valid'],
              'verbose': 0,
              'categorical_feature': 'auto'}

param_test = {'learning_rate' : [0.01, 0.02, 0.03, 0.04],
              'n_estimators' : [5, 10 , 100, 200, 300],
              'num_leaves': sp_randint(6, 50),
              'min_child_samples': sp_randint(100, 500),
              'min_child_weight': [1e-5, 1e-3, 1e-2, ],
              'subsample': sp_uniform(loc=0.2, scale=0.8),
              'max_depth': [-1, 1, 2, 3, 4, 5, 6, 7],
              'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
              'reg_alpha': [0, 1e-1, 1, 2],
              'reg_lambda': [0, 1e-1, 1, 5]}

#number of combinations
n_iter = 100

#intialize lgbm and lunch the search
lgbm_clf = lgbm.LGBMClassifier(random_state=random_state, silent=False, metric='None', n_jobs=4)
grid_search = RandomizedSearchCV(
    estimator=lgbm_clf, param_distributions=param_test,
    n_iter=n_iter,
    scoring='accuracy',
    cv=4,
    refit=True,
    random_state=random_state,
    verbose=True)

grid_search.fit(X_res, y_res, **fit_params)
opt_parameters =  grid_search.best_params_
lgbm_clf = lgbm.LGBMClassifier(**opt_parameters)

Fitting 4 folds for each of 100 candidates, totalling 400 fits


KeyboardInterrupt: 

In [79]:
# get a stacking ensemble of models
def get_stacking():
    # define the base models
    level0 = list()
    level0.append(('lr', LogisticRegression(C=0.0001, class_weight = 'balanced',max_iter=200, solver = 'saga',penalty = 'elasticnet', l1_ratio = 0.01)))
    level0.append(('knn', KNeighborsClassifier()))
    level0.append(('bbc', BalancedBaggingClassifier()))
    level0.append(('svm', SVC()))
    level0.append(('bayes', GaussianNB()))
    # level0.append(('lama', automl))
    level0.append(('lgbm', lgbm_clf))
    # define meta learner model
    level1 = LogisticRegression(C=0.0001, class_weight = 'balanced',max_iter=200, solver = 'saga',penalty = 'elasticnet', l1_ratio = 0.01)
    # define the stacking ensemble
    model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
    return model

In [54]:
def get_kaggle_results():
    p = subprocess.Popen("kaggle competitions submissions  -c home-credit-default-risk", stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    print(output.decode())

In [80]:
stacking = get_stacking()

In [81]:
stacking.fit(X_res, y_res)

StackingClassifier(cv=5,
                   estimators=[('lr',
                                LogisticRegression(C=0.0001,
                                                   class_weight='balanced',
                                                   l1_ratio=0.01, max_iter=200,
                                                   penalty='elasticnet',
                                                   solver='saga')),
                               ('knn', KNeighborsClassifier()),
                               ('bbc', BalancedBaggingClassifier()),
                               ('svm', SVC()), ('bayes', GaussianNB()),
                               ('lgbm',
                                LGBMClassifier(boosting_type='goss',
                                               class_weight='balanced',
                                               eval_set=[        SK_ID_CURR...
171000                           1  
221813                           1  
270975                           0  

[6

In [ ]:
predict_2 = stacking.predict(X_test)

In [ ]:
write_submission_2 = pd.read_csv('data/sample_submission.csv')
write_submission_2['TARGET'] = predict_2
write_submission_2.to_csv('predict_stacking.csv', index= False)

In [ ]:
!kaggle competitions submit -c home-credit-default-risk -f predict_stacking.csv -m "lgbm"

In [ ]:
get_kaggle_results()